# Import Libraries & Dependencies

In [1]:
import math
import random
import pickle
import statistics
import numpy as np

# Reading Data

In [2]:
# Read variables from file
with open("params.pkl", "rb") as f:
    
    variable_ranges = pickle.load(f)
    bits_per_variable = pickle.load(f)
    num_variables = pickle.load(f)
    pop_size = pickle.load(f)
    pc = pickle.load(f)
    pm = pickle.load(f)
    max_generations = pickle.load(f)
    num_runs = pickle.load(f)

In [3]:
print("variable_range:", variable_ranges)
print("bits_per_variable:", bits_per_variable)
print("num_variables:", num_variables)
print("pop_size:", pop_size)
print("pc:", pc)
print("pm:", pm)
print("max_generations:", max_generations)
print("num_runs:", num_runs)

variable_range: [[-7.0, 4.0], [-7.0, 4.0], [-7.0, 4.0]]
bits_per_variable: [15, 20, 25]
num_variables: 3
pop_size: 50
pc: 0.9
pm: 0.05
max_generations: 100
num_runs: 50


# Population Initialization

In [5]:
def initialize_population():
    # Initialize the population with random bit-strings for each variable
    population = []
    for i in range(pop_size):
        x1_bitstring = ''.join(random.choices(['0', '1'], k=bits_per_variable[0]))
        x2_bitstring = ''.join(random.choices(['0', '1'], k=bits_per_variable[1]))
        x3_bitstring = ''.join(random.choices(['0', '1'], k=bits_per_variable[2]))
        individual = x1_bitstring + x2_bitstring + x3_bitstring
        population.append(individual)
    return population

# Decoding Individuals

In [6]:
def decode_individual(individual):
    # Convert bit-strings to floating point values
    x1 = variable_ranges[0][0] + (int(individual[:bits_per_variable[0]], 2) / (2**bits_per_variable[0]-1)) * (variable_ranges[0][1] - variable_ranges[0][0])
    x2 = variable_ranges[1][0] + (int(individual[bits_per_variable[0]:bits_per_variable[0]+bits_per_variable[1]], 2) / (2**bits_per_variable[1]-1)) * (variable_ranges[1][1] - variable_ranges[1][0])
    x3 = variable_ranges[2][0] + (int(individual[-bits_per_variable[2]:], 2) / (2**bits_per_variable[2]-1)) * (variable_ranges[2][1] - variable_ranges[2][0])
    return [x1, x2, x3]

# Evaluating Population

In [7]:
def evaluate_population(population):
    # Evaluate the fitness of each individual in the population
    fitness_values = []
    for individual in population:
        [x1, x2, x3] = decode_individual(individual)
        fitness = x1**2 + x2**2 + x3**2
        fitness_values.append(fitness)
    return fitness_values

# Proportional Selection

In [8]:
def proportional_selection(population, fitness_values):
    # Perform proportional selection
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    selected_indices = random.choices(range(pop_size), weights=probabilities, k=pop_size)
    selected_population = [population[i] for i in selected_indices]
    return selected_population

# Single - Point Crossover

In [9]:
def single_point_crossover(parent1, parent2):
    # Perform single-point crossover
    crossover_point = random.randint(0, len(parent1))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Mutation

In [10]:
# Define the mutation function
def mutation(individual):
    # Perform gene-wise mutation
    x1_list = list(individual[:bits_per_variable[0]])
    x2_list = list(individual[bits_per_variable[0]:bits_per_variable[0]+bits_per_variable[1]])
    x3_list = list(individual[-bits_per_variable[2]:])
    for i in range(len(x1_list)):
        if random.random() < pm:
            x1_list[i] = '1' if x1_list[i] == '0' else '0'
    for i in range(len(x2_list)):
        if random.random() < pm:
            x2_list[i] = '1' if x2_list[i] == '0' else '0'
    for i in range(len(x3_list)):
        if random.random() < pm:
            x3_list[i] = '1' if x3_list[i] == '0' else '0'
    individual = ''.join(x1_list + x2_list + x3_list)
    return individual


# # Defining the GA function

In [11]:
def run_ga():
    # Initialize the best solution and its fitness value
    best_of_run_solution = None
    best_of_run_fitness = float("inf")

    # Run the GA for 30 independent runs
    best_fitnesses = []
    
    for run in range(num_runs):
        # Initialize the population
        population = initialize_population()

        # Run the GA for the specified number of generations
        for gen in range(max_generations):
            # Evaluate the fitness of the population
            fitness_values = evaluate_population(population)

            # Select the parents for the next generation
            selected_population = proportional_selection(population, fitness_values)

            # Create a new population through crossover and mutation
            new_population = []
            
            for i in range(int(pop_size / 2)):
                # Select two parents at random from the selected population
                [parent1, parent2] = random.sample(selected_population, k=2)

                # Perform crossover with probability pc
                if random.random() < pc:
                    [child1, child2] = single_point_crossover(parent1, parent2)
                else:
                    child1, child2 = parent1, parent2

                # Perform mutation with probability pm
                child1 = mutation(child1)
                child2 = mutation(child2)

                # Add the new children to the new population
                new_population.append(child1)
                new_population.append(child2)

            # Replace the old population with the new populationa
            population = new_population

            # Track the best fitness of the population
            best_fitness = min(fitness_values)
            best_solution = population[np.argmin(fitness_values)]

            # Update the best solution and fitness value if a better solution is found
            if best_fitness < best_of_run_fitness:
                best_of_run_solution = best_solution
                best_of_run_fitness = best_fitness

        # Record the best fitness of the run
        best_fitnesses.append(best_fitness)
        
    return best_of_run_solution, best_of_run_fitness, best_fitnesses

In [12]:
# Run the GA for the specified number of independent runs
best_solution, best_fitness , best_fitnesses = run_ga()

In [13]:
# Print the average and standard deviation of the best fitness over all runs
avg_best_fitness = statistics.mean(best_fitnesses)
std_best_fitness = statistics.stdev(best_fitnesses)
print(f"\nAverage best fitness over {num_runs} runs: {avg_best_fitness:.4f}")
print(f"Standard deviation of best fitness over {num_runs} runs: {std_best_fitness:.4f}")

# Print the vector of the best-of-run solution
print(f"\nBest-of-run solution:")
print(f"Bit-string representation: {best_solution}")
x, y, z = decode_individual(best_solution)
print(f"Floating-point values: x1 = {x:.4f}, x2 = {y:.4f}, x3 = {z:.4f}")
print(f"Fitness value: {best_fitness:.4f}")


Average best fitness over 50 runs: 26.1582
Standard deviation of best fitness over 50 runs: 12.8224

Best-of-run solution:
Bit-string representation: 001010010001111110010001001100100100000000010110001100010111
Floating-point values: x1 = -5.2332, x2 = 1.6195, x3 = -6.9702
Fitness value: 0.0815
